# Parsing the notebook

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import display, Markdown, HTML
from pprint import pprint

## Loading a notebook

In [ ]:
#|default_exp nbparser
#|export
from typing import Union, List, Tuple
from nbx.utils import listmap
import ipynbname
import json
from nbx.utils import Bunch
from pathlib import Path

In [ ]:
#|export
def load_nb(fname):
    """Loads a a ipynotebook-bunch""" 
    try:
        nbdict = json.loads(fname)
    except:
        nbdict = json.load(open(fname,'r',encoding="utf-8"))
        
    nbdict["fname"] = str(fname)
    return Bunch(**nbdict)

In [ ]:
# %%capture
nb = load_nb("nbx_example/test_notebook.ipynb")
# for i,cell in enumerate(nb.cells):
#     h = f"|   cell {i}   |"
#     n = len(h) - 2    
#     display(HTML(f"<h3 style='color: black'>Cell - {i} - {cell['cell_type']}</h3>"))
#     s = "<br>".join(cell["source"])
#     s = f"<code>{s}</code>"
#     display(HTML(s.replace("\n", "<span style='color:red'> &crarr;</span>")))

# Parsing a line

In [ ]:
# #|export 
# def add_fail(p):
#     """
#     Adds an argument that determines the 
#     return value in case of a failed parse
#     """
#     def q(*args, fail=None, **kwargs):
#         r = p(*args, **kwargs, )
#         if r is None: return fail
#         else: return r
#     return q

## Parsing Hashtags and Commands (`#nbx`)

In [ ]:
#|export
import re
tag_rx = re.compile(r"^\s*#([a-zA-Z_]+)\s*(.*)$")

def parse_any_tag(line):
    """
    Splits off a tag (eg. `#MyTAg`) at the beginning of the line. 
    """
    m = tag_rx.match(line)
    if m is not None: return m.groups()
    else: return None

In [ ]:
parse_any_tag("#nbx something else")

('nbx', 'something else')

In [ ]:
#|export
def parse_tag(name):
    """Splits off a specified tag only"""

    def p(line):
        r = parse_any_tag(line)
        if r is None or r[0] != name: return None
        else: return r
    return p

In [ ]:
assert parse_tag("nbx")("#nbx")[0]    == "nbx"
assert parse_tag("nbx")("#nbx ")[0]   == "nbx"
assert parse_tag("nbx")("#nbx: ")[0]  == "nbx"
assert parse_tag("nbx")("#nbx : ")[0] == "nbx"
assert parse_tag("nbx")("#xarg  something else ") == None

In [ ]:
#|export
def result_or_none(y:"returned object by a parser"):
    if y is None: return None
    else: return y[0]

In [ ]:
assert result_or_none(parse_any_tag("#nbx")) == "nbx"
assert result_or_none(parse_any_tag("#nbx ")) == "nbx"
assert result_or_none(parse_any_tag("# nbx something else ")) == None

## Parsing flags  (`#nbx --fname script.py --src ./src`)

In [ ]:
# #|export
# import re
# flag_rx = re.compile(r"(?:\-\-|\-)(?!\-)(?P<name>[a-zA-Z_\-]+)\s*(?P<rest>.*)")

# def flagname(line):
#     """
#     Splits off the name of a flag `--flagname`. 
#     """
#     m = flag_rx.match(line)
#     if m is not None: 
#         d = m.groupdict()
#         return (d["name"], d["rest"])
#     else: return None
    
    
# def is_flag(line): return flagname(line) is not None

In [ ]:
# assert flagname("--flag-name asdf") == ('flag-name', 'asdf')
# assert flagname("--flag_name asdf") == ('flag_name', 'asdf')

In [ ]:
# #|export
# import shlex
# from collections import defaultdict

# def flags(line):
#     d = defaultdict(list)
#     n = None
#     for s in shlex.split(line):
#         if is_flag(s): 
#             n,_ = flagname(s)
#             d[n] = []
#         else: d[n].append(s.strip())
#     return (dict(d), "")

In [ ]:
# flags("--fname experiment.py  -s ./src ")

In [ ]:
# shlex.split('--fname="experiment.py"  -s ./src -o 1')

In [ ]:
#|export
import re
# flag_rx =     re.compile(r"(?:[\-]+)(?P<name>[a-zA-Z_\-]+)=?(.*)$")
flagname_rx = re.compile(r"(?:[\-]+)(?P<name>[a-zA-Z_\-]+)=?(.*)?$")
def flagname(line):
    """
    Splits off the name of a flag `--flagename`. 
    """
    m = flagname_rx.match(line)
    if m is not None: return m.groups()[0]
    else: return None
    
# def is_flag(line): return flagname(line) is not None

# def pairs(arr):
#     n = len(arr); 
#     assert n%2==0
#     ps = []
#     for i in range(n//2): ps.append((arr[2*i],arr[2*i+1]))
#     return ps

In [ ]:
# is_flag("-flag_name"), flagname("--some-name=bla"), flagname("-some_other_name")

In [ ]:
# opt_rx = re.compile(r"(?P<key>\-\-[a-zA-Z_\-]+)=(?P<value>.*)?$")
# def split_option_assignment(x):
#     m = opt_rx.match(x)
#     if m is not None: return m.groups()
#     else: return [x]
    
# split_option_assignment("--src=2")

In [ ]:
#|export
import re
flagname_rx = re.compile(r"(?:[\-]+)(?P<name>[a-zA-Z_\-]+)=?(.*)?$")

def parse_flagname(line):
    m = flagname_rx.match(line)
    if m is not None: 
        n,r = m.groups()
        if r is None: r = ""
        return (n,r)
    else: return None

In [ ]:
flagname("--src=src")
# assert flagname("--src=src") == "src"

'src'

In [ ]:
# from collections import defaultdict
# import shlex 

# def parse_flags(line):
#     d    = dict()
#     last = None
#     rest = [] # not a flag-key nor a value 
#     for s in shlex.split(line):
#         try:
#             n, v = parse_flagname(s)
#             if n not in d: d[n] = []
#             if v != "": d[n].append(v.strip())
#             last = None
#         except: 
#             if last is not None: d[last].append(s.strip())
#             else: rest.append(s)
#             last = None
#     return (d, rest)


In [ ]:
# parse_flags('--some-name=A B C --something_else=D --some-name=E --fname')

In [ ]:
from collections import defaultdict
import shlex 

def parse_flags(line):
    d    = dict()
    last = None
    rest = [] # not a flag-key nor a value 
    for s in shlex.split(line):
        try:
            n, v = parse_flagname(s)
            v_ = v.strip()
            if v_ == "":
                
                d[n]= None
                last = n
            else:
                d[n] = v_
                last = None
        except: 
            if last is not None: 
                d[last] = s
                last = None
            else:
                rest.append(s)
         
    return (d, rest)

In [ ]:
parse_flags('--some-name=A B C --something_else=D --some-name=E --fname')


({'some-name': 'E', 'something_else': 'D', 'fname': None}, ['B', 'C'])

In [ ]:
# #|export
# import shlex
# def parse_flags(line):    
    
#     args = shlex.split(line)
#     if len(args) % 2 != 0: return None
    
#     d = {}
#     for k,v in pairs(args):
#         if not is_flag(k): return None
#         d[flagname(k)] = v
    
#     return (d, "")


In [ ]:
# parse_flags('--src=./src --fname experiment.py')

In [ ]:
#|export
import shlex
def parse_nbx_meta(line):    
    _, r = parse_tag("nbx_meta")(line)
    try:
        f, r = parse_flags(r)
        f["tag"] = "nbx_meta"
    except: return None
    return (f,r)


In [ ]:
line = '--src=./src --fname=experiment.py'

In [ ]:
parse_nbx_meta("#nbx_meta --fname test_script.py --src ./src")

({'fname': 'test_script.py', 'src': './src', 'tag': 'nbx_meta'}, [])

In [ ]:
print(parse_nbx_meta("#nbx_meta  --fname experiment.py  -s ./src  "))
print(parse_nbx_meta("#nbx_meta  experiment.py  "))
print(parse_nbx_meta("#nbx_meta -n  experiment.py  --src src "))

({'fname': 'experiment.py', 's': './src', 'tag': 'nbx_meta'}, [])
({'tag': 'nbx_meta'}, ['experiment.py'])
({'n': 'experiment.py', 'src': 'src', 'tag': 'nbx_meta'}, [])


## Parsing `xarg`

In [ ]:
#|export
xarg_rx = re.compile(r"""
^(?P<name>[^=]+)=(?P<val>[^;#]+)
(?:(?:;(?P<sweep>[^\#]*))?(?:\#(?P<help>.*))?)?
$""", re.VERBOSE)

def strip(s):
    return s.strip()

def strip_or_none(s):
    if s is None: return None
    else: return s.strip()

def strip_dict_vals(d):
    for k,v in d.items():
        d[k] = strip_or_none(v)
    return d
    
def parse_xarg(line):
    """
    Parses the line below an `xarg` tag and
    returns a dict with keys `[name,val,type,help,sweep]`.
    """

    m = xarg_rx.match(line)
    if m is None: return None
    
    d = dict(**m.groupdict())
    d = strip_dict_vals(d)
    
    v = eval(d['val'])
    d['type'] = type(v).__name__ 
    
    if d['help']  == "": d['help']  = None
    if d['sweep'] == "": d['sweep'] = None
    
    return (d, "")


In [ ]:
test_lines = [
    ("x + 2 ; [] ",      None),
    ("x = 2  ",      ("x","2",None,None)),
    ("x = 2  ;  ",   ("x","2",None,None)),
    ("x = 2  ;  # ", ("x","2",None,None)),
    ("x = 'hello' ; range(2)   # some description  ", ("x", "'hello'", "range(2)", "some description")),
    ("x = 2 ;  [f(1),f(a),6,8]  ",                    ("x", "2", "[f(1),f(a),6,8]", None)),
    ("x = 'hello'    ; # some description  ",         ("x", "'hello'", None, "some description")),
    ("x = 'hello'   # some description  ",            ("x", "'hello'", None, "some description")),
]   

for line, expected in test_lines:
    r = result_or_none(parse_xarg(line))
    print(r)
    if r is None: 
        assert r == expected
        continue
    assert (r["name"], r["val"], r["sweep"], r["help"]) == expected 

None
{'name': 'x', 'val': '2', 'sweep': None, 'help': None, 'type': 'int'}
{'name': 'x', 'val': '2', 'sweep': None, 'help': None, 'type': 'int'}
{'name': 'x', 'val': '2', 'sweep': None, 'help': None, 'type': 'int'}
{'name': 'x', 'val': "'hello'", 'sweep': 'range(2)', 'help': 'some description', 'type': 'str'}
{'name': 'x', 'val': '2', 'sweep': '[f(1),f(a),6,8]', 'help': None, 'type': 'int'}
{'name': 'x', 'val': "'hello'", 'sweep': None, 'help': 'some description', 'type': 'str'}
{'name': 'x', 'val': "'hello'", 'sweep': None, 'help': 'some description', 'type': 'str'}


In [ ]:
print(parse_xarg("x = 2 ;  [f(1),f(a),6,8]  "))
print(parse_xarg("x = 'hello' ; range(2)   # some description  "))
print(parse_xarg("x = 'hello' ; # some description  "))

({'name': 'x', 'val': '2', 'sweep': '[f(1),f(a),6,8]', 'help': None, 'type': 'int'}, '')
({'name': 'x', 'val': "'hello'", 'sweep': 'range(2)', 'help': 'some description', 'type': 'str'}, '')
({'name': 'x', 'val': "'hello'", 'sweep': None, 'help': 'some description', 'type': 'str'}, '')


# Parsing a cell

In [ ]:
#|export
def parse_default(line):
    return (line.rstrip(), "")

In [ ]:
#|export 
def consume_line(tag, p = parse_default):
    """
    Returns a parser that 
    consume a line and parses it.
    """
    
    def q(a, src):
        if len(src) == 0: return a, []
        line = src[0]
        
        b = result_or_none(p(line))
        
        a.append((tag, line, b))
        return a, src[1:]
    
    return q

In [ ]:
#|export 
def consume_block(tag, p = parse_default):
    """
    Returns a parser that consume a block of lines 
    and parse them, till you hit an empty line, 
    a comment/tag, or the end.
    """
    
    def parse(a, src):
        if len(src) == 0: return a, []
        
        line = src[0]
        
        # stopping conditions
        if line.strip() == "": return a, src
        if line.strip()[0] == "#":  return a, src
        
        # else consume line
        b = result_or_none(p(line))        
        a.append((tag, line, b))

                 
        return parse(a, src[1:])
                 
    return parse



In [ ]:
#|export 
def ignore_first(p):
    """
    Return a parser that ignores the first line and 
    starts parsing starting at the next.
    """
    def q(a, src):
        if len(src) <= 1: return a, []
        else: return p(a, src[1:])
    return q


In [ ]:
#|export 
def consume_line_below(tag, p = parse_default):
    return ignore_first(consume_line(tag, p=p))

def consume_block_below(tag, p = parse_default):
    return ignore_first(consume_block(tag, p=p))

In [ ]:
#|export 
def parse_src(a:list, src:list, parser_dict: dict, parse_default=parse_default):
    """Parses a cell sources..."""
    if len(src) == 0: return a, []
    
    line = src[0]
    
    tag = result_or_none(parse_any_tag(line))
    
    if tag is None or tag not in parser_dict:
        b, _ = parse_default(line)
        a.append((None, line, b))
        rest = src[1:]
    else:        
        p = parser_dict[tag]
        a, rest = p(a, src)

    return parse_src(a, rest, parser_dict)

In [ ]:
#|export 
PARSER_DICT = {
    'nbx'       : consume_line('nbx'),
    'nbx_meta'  : consume_line('nbx_meta', parse_nbx_meta),
    'nbx_return': consume_line_below("nbx_return"), 
    'xarg' : consume_block_below('xarg', parse_xarg),
    'xargs': consume_block_below('xarg', parse_xarg),
    'ximp' : consume_block_below('ximp'),
    'ximps': consume_block_below('ximps')}

def parse_cell(cell, parser_dict=PARSER_DICT):
    src = listmap(lambda line: line.rstrip(), cell['source'])
    a, _ = parse_src([], src, parser_dict)
    return a

In [ ]:
for i,cell in enumerate(nb.cells):    
    print(f"CELL {i: >2.0f} PARSED: ( type | line | result )")
    print(f"----------------------------------------")

    a = parse_cell(cell)    
    for tag, line, res in a:
        t = f"[{tag}]"
        dots = " ...\"" if len(line) > 25 else "\""
        print(f"{t:<15s} | \"{line[:35]+dots:<40s} | {res}")
    print("\n")

CELL  0 PARSED: ( type | line | result )
----------------------------------------
[None]          | "# Test Notebook for `nbx`"               | # Test Notebook for `nbx`


CELL  1 PARSED: ( type | line | result )
----------------------------------------
[None]          | "%load_ext autoreload"                    | %load_ext autoreload
[None]          | "%autoreload 2"                           | %autoreload 2
[None]          | "%matplotlib inline"                      | %matplotlib inline


CELL  2 PARSED: ( type | line | result )
----------------------------------------
[nbx]           | "#nbx"                                    | #nbx
[nbx_meta]      | "#nbx_meta --fname test_script.py -- ..." | {'fname': [], 'src': [], 'tag': 'nbx_meta'}
[None]          | ""                                        | 
[None]          | "print("This message is in the noteb ..." | print("This message is in the notebook and will be added")
[None]          | ""                                        | 
[x

# Parsing the whole thing

In [ ]:
#|export
def is_none(x): return x is None
def is_not_none(x): return x is not None
def is_tagged(name):
    return lambda line: is_not_none(parse_tag(name)(line))
is_nbx = is_tagged("nbx")

In [ ]:
assert is_nbx("#nbx " ) 
assert is_nbx("# nbx" ) == False
assert is_nbx(" #nbx ") 
assert is_nbx("  #nbx") 

In [ ]:
#|export 
def is_nbx_cell(cell):
    if cell['cell_type'] != 'code': return False
    if not cell['source']: return False
    line0 = cell['source'][0]
    return is_nbx(line0)

In [ ]:
#|export
magic_shell_rx =  re.compile(r"^\s*%{1,2}|^\s*!")

def is_magic_or_shell(line):
    """Checks if line contains a jupyter 
    magic function or shell command"""
    m = magic_shell_rx.match(line)
    return m is not None

In [ ]:
assert is_magic_or_shell("%pwd ")
assert is_magic_or_shell("%%capture ")
assert is_magic_or_shell("!ls")

In [ ]:
#|export
from collections import defaultdict

def move_to_front(arr, i):
    return [arr[i]] + arr[:i] + arr[i+1:]
    
XARG_MISSING = """
        
#xargs
xtask = 0
xdir  = './'

"""

def parse_nb(nb, parser_dict=PARSER_DICT):
    """Parses a notebook bunch"""
    
    # Parse all of the `nbx` cells
    nbx_cells = filter(is_nbx_cell, nb.cells)
    A = defaultdict(list)
    for cell in nbx_cells:
        a = parse_cell(cell, parser_dict=PARSER_DICT)

        for t, line, b in a:
            if t is None: t = 'func_body'
            A[t].append(b)
        
    # Make sure xtask and xdir are present
    argdict = {}
    for i,x in enumerate(A["xarg"]): argdict[x['name']] = i
    if "xdir" not in argdict or "xtask" not in argdict:
        raise NameError("YOU NEED TO ADD X-ARGS `xdir` and `xtask`(see below):..."+XARG_MISSING)
    move_to_front(A["xarg"], argdict["xdir"])
    move_to_front(A["xarg"], argdict["xtask"])
    
    # Parameter sweeps
    A["sweep"]     = []
    A["non_sweep"] = []
    for x in A["xarg"]:
        if x["sweep"] is None: 
            A["non_sweep"].append(x)
        else: 
            A["sweep"].append(x)
            
        
    # Meta data  
    if "nbx_meta" in A: A["nbx_meta"] = A["nbx_meta"][0]
    else: A["nbx_meta"] = {}
    A["nbx_meta"]["nbname"] = nb["fname"]
            
    # Set the return value
    if "nbx_return" in A:
        A["nbx_return"] = A["nbx_return"][-1]

    return Bunch(**A)

In [ ]:
nb  = load_nb("nbx_example/test_notebook.ipynb")
pnb = parse_nb(nb)
print(f"Keys NB:\n\t{nb.keys()}")
print(f"Keys Parsed NB:\n\t{pnb.keys()}")

Keys NB:
	dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor', 'fname'])
Keys Parsed NB:
	dict_keys(['nbx', 'nbx_meta', 'func_body', 'xarg', 'ximp', 'nbx_return', 'sweep', 'non_sweep'])


In [ ]:
pnb.func_body

['',
 'print("This message is in the notebook and will be added")',
 '',
 '',
 '',
 'print("This message is in the notebook and will be added")',
 '',
 '',
 '',
 '',
 '',
 '',
 'def f():',
 '    pass',
 '',
 '',
 '',
 'summed = sum([y,z,w_prime])',
 '',
 'print("Cell 1", summed)',
 'import sys',
 'sys.dont_write_bytecode = False',
 'from src.other_stuff import INFO',
 'print(INFO)',
 'summed = summed*0.1',
 'print("Cell 2", summed)',
 '']

# Creating the nbx bundle

In [ ]:
#|export
from pathlib import PurePosixPath as Path
import pkg_resources
import importlib
from nbx.utils import nbx_lib
from nbx.templ import *
import os

def add_if_necessary(d, k, v):
    if k not in d:
        d[k] = v
        
TEMPLATE_PATH = Path("/Users/mirko/Workspace/nbx/notebooks/nbx_example/templates")
TEMPLATE_PATH 

PurePosixPath('/Users/mirko/Workspace/nbx/notebooks/nbx_example/templates')

In [ ]:
#|export 
def create_script(fname, tpl, vdict={}):
    """Create script from template and value dict""" 
    return create_file_from_template(TEMPLATE_PATH/tpl, fname, vdict)

## `experiment.py`

In [ ]:
#|export 
def create_experiment(pnb, fname="experiment.py", tpl="experiment.tpl"):
    return create_script(fname, tpl, pnb)

## `confspace.yaml`

In [ ]:
#|export
from nbx.fileio import dump, load

def create_confspace(pnb, fname="confspace.json"):
    meta = {"nbname": str(pnb["nbx_meta"]["nbname"])}
    C = dict(info=meta, src=dict(), default_src=dict())
    default = {}
    for x in pnb["xarg"]:
        key, val, sweep = x["name"], x["val"], x["sweep"]
        C["default_src"][key] = val
        if sweep is None: sweep = f"[{val}]"
        C["src"][key] = sweep
    dump(C, str(fname))
    
    return C

## `wrapper.py`

In [ ]:
#|export    
def create_wrapper(pnb, 
                   fname = "wrapper.py", 
                   xname = "experiment.py", 
                   confspace = "confspace.json", 
                   tpl = "wrapper.tpl"):
    
    stem = Path(xname).stem
    return create_script(fname, tpl, 
                         vdict = dict(module=stem, confspace=confspace))

## `runner.sh`

In [ ]:
#|export
def create_runner(fname="runner.sh", wrapper="wrapper.py", tpl="run.tpl"):
    return create_script(fname, tpl, vdict=dict(wrapper=wrapper))

## `nbx_bundle/`

In [ ]:
#|export
import shutil
from nbx.utils import run_bash
from importlib import import_module
import sys

def create_bundle(xdir, nbpath=None):
    """
    Creates a nbx-bundle at the given directory
    from a notebook.
    
    The notebook is either specified by its path, or 
    in case of "None" created from the notebook 
    the function is called from.
    """
    
    if not os.path.exists(xdir): os.makedirs(xdir)
    if nbpath is None: nbpath = ipynbname.path()
    xdir, nbpath = Path(xdir), Path(nbpath)
    
    pnb = parse_nb(load_nb(nbpath))

    create_script(xdir/"__init__.py", "init.tpl", vdict={})
    create_experiment(pnb, fname=xdir/"experiment.py", tpl="experiment.tpl")
    create_confspace( pnb, fname=xdir/"confspace.json")
    create_wrapper(   pnb, fname=xdir/"wrapper.py", xname="experiment.py", confspace="confspace.json", tpl="wrapper.tpl")
    create_runner(fname=xdir/"runner.sh", wrapper="wrapper.py", tpl="run.tpl")
    
    # Btw: -B suppresses the creation of __pycache__
    num_confs, = run_bash(f"python -B {xdir/'wrapper.py'} info")
    num_confs = int(num_confs.strip())

    print(pnb["nbx_meta"])
    print(f"Creating bundle:\n")
    if "src" in pnb["nbx_meta"]:
        xsrc = pnb["nbx_meta"]["src"]
        if os.path.exists(nbpath.parent/xsrc): 
            print(f"\t{xdir/xsrc/'*'}")  
            
            if os.path.exists(str(xdir/xsrc)): 
                shutil.rmtree(str(xdir/xsrc))    
                
            shutil.copytree(str(nbpath.parent/xsrc), str(xdir/xsrc))
            
            
    print(f"\t{xdir/'experiment'}")    
    print(f"\t{xdir/'confspace.json'} (#confs = {num_confs})")
    print(f"\t{xdir/'wrapper.py'}")
    print(f"\t{xdir/'runner.sh'}\n")
    


In [ ]:
!rm -r nbx_example/_nbx
create_bundle("nbx_example/_nbx", "nbx_example/test_notebook.ipynb")
!ls nbx_example/_nbx

{'fname': 'test_script.py', 'src': './src', 'tag': 'nbx_meta', 'nbname': 'nbx_example/test_notebook.ipynb'}
Creating bundle:

	nbx_example/_nbx/src/*
	nbx_example/_nbx/experiment
	nbx_example/_nbx/confspace.json (#confs = 18)
	nbx_example/_nbx/wrapper.py
	nbx_example/_nbx/runner.sh

__init__.py    experiment.py  src
confspace.json runner.sh      wrapper.py
